<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-предобработка" data-toc-modified-id="Загрузка-и-предобработка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и предобработка</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Мешок-слов" data-toc-modified-id="Мешок-слов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Мешок слов</a></span><ul class="toc-item"><li><span><a href="#Downsample" data-toc-modified-id="Downsample-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Downsample</a></span></li></ul></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>TF-IDF</a></span><ul class="toc-item"><li><span><a href="#Downsample" data-toc-modified-id="Downsample-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Downsample</a></span></li></ul></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span><ul class="toc-item"><li><span><a href="#CountVectorizer" data-toc-modified-id="CountVectorizer-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>CountVectorizer</a></span></li><li><span><a href="#TfidfVectorizer" data-toc-modified-id="TfidfVectorizer-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>TfidfVectorizer</a></span></li></ul></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>CatBoostClassifier</a></span><ul class="toc-item"><li><span><a href="#CountVectorizer" data-toc-modified-id="CountVectorizer-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>CountVectorizer</a></span></li><li><span><a href="#TfidfVectorizer" data-toc-modified-id="TfidfVectorizer-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>TfidfVectorizer</a></span></li></ul></li><li><span><a href="#LightGBMClissifier" data-toc-modified-id="LightGBMClissifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>LightGBMClissifier</a></span><ul class="toc-item"><li><span><a href="#CountVectorizer" data-toc-modified-id="CountVectorizer-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>CountVectorizer</a></span></li><li><span><a href="#TfidfVectorizer" data-toc-modified-id="TfidfVectorizer-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>TfidfVectorizer</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span><ul class="toc-item"><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

**Описание проекта**  

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.  
Постройте модель со значением метрики качества F1 не меньше 0.75.

**Описание данных**

`text` - содержит текст комментария  
`toxic` - целевой признак

In [1]:
 !pip install nltk


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install catboost


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np


import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from catboost import Pool, CatBoostClassifier
import lightgbm as lgb

import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\79119\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79119\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\79119\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
pd.set_option('max_colwidth', None)

In [5]:
stemmer = WordNetLemmatizer()

In [6]:
stop_words = stopwords.words('english')
stop_words.remove('not')

## Загрузка и предобработка

In [7]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv(r'C:\Users\79119\Desktop\DF\comments\toxic_comments.csv')


In [8]:
def info_df(df):
    print('Ознакомление')
    display(df.head())
    print(' ')
    print('Общая информация')
    print(df.info())
    print(' ')
    print('Дубликаты')
    print(df.duplicated().sum())
    print(' ')
    print('Пропуски')
    print(df.isna().sum())
    print(' ')
    print('Проверка баланса классов')
    print(df['toxic'].value_counts(normalize=True))

In [9]:
info_df(df)

Ознакомление


,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


 
Общая информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None
 
Дубликаты
0
 
Пропуски
Unnamed: 0    0
text          0
toxic         0
dtype: int64
 
Проверка баланса классов
0    0.898388
1    0.101612
Name: toxic, dtype: float64


1. Удалим все лишние знаки и пробелы.  
2. Приведем к нижнему регистру. 
4. Проведем токенизацию
3. Произведем лемматизацию.

In [10]:
def data_processing(text):
    text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    text = [word for word in tokens if word not in stop_words]
    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [11]:
df['lemm_text'] = df['text'].apply(lambda text: data_processing(text))

In [12]:
df.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation edits made username hardcore metallica fan reverted vandalism closure gas voted new york doll fac please remove template talk page since retired 89 205 38 27
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,aww match background colour seemingly stuck thanks talk 21 51 january 11 2016 utc
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man really not trying edit war guy constantly removing relevant information talking edits instead talk page seems care formatting actual info
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,make real suggestion improvement wondered section statistic later subsection type accident think reference may need tidying exact format ie date format etc later one else first preference formatting style reference want please let know appears backlog article review guess may delay reviewer turn listed relevant form eg wikipedia good article nomination transport
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0,sir hero chance remember page


In [47]:
print(df.shape)
print( )
print(df.isna().sum())
print( )
print(df.duplicated().sum())

(159292, 4)

Unnamed: 0    0
text          0
toxic         0
lemm_text     0
dtype: int64

0


**Вывод:**  
1. Нам представлен датафрейм, состоящий из 159571 строк и 2 столбцов. Столбец `text` содержит комментарии, столбец `toxic` содержит целевой признак.  
2. При анализе выявлени дисбаланс классов в соотношении 9:1  
3. В данных нет ни дубликатов, ни прпусков.
4. В столбце `lemm_text` размещены очищенные от лишних знакои и пробелов и лемматизированные комментарии для дальнейшей работы.

## Подготовка данных

In [14]:
X = df['lemm_text']
y = df['toxic']

X.shape

(159292,)

In [15]:
X_train, X_valid_test, y_train, y_valid_test = train_test_split(
    X, y, 
    test_size = 0.3, 
    random_state=42, 
    stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_valid_test,
    y_valid_test,
    test_size=0.5,
    random_state=42,
    stratify=y_valid_test
)
print(f"Количество строк в y_train по классам: {np.bincount(y_train)}")
print(f"Количество строк в y_valid по классам: {np.bincount(y_valid)}")
print(f"Количество строк в y_test по классам: {np.bincount(y_test)}")

Количество строк в y_train по классам: [100174  11330]
Количество строк в y_valid по классам: [21466  2428]
Количество строк в y_test по классам: [21466  2428]


### Мешок слов

In [16]:
count_vect = CountVectorizer()
X_train_vect = count_vect.fit_transform(X_train)
X_valid_vect = count_vect.transform(X_valid)
X_test_vect = count_vect.transform(X_test)

In [17]:
(X_train_vect.shape[0] + X_valid_vect.shape[0] + X_test_vect.shape[0])- X.shape[0]

0

#### Downsample

Чтобы сбалансировать целевой признак, уменьшим выборку.

In [18]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=42)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=42)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled,
                                                       target_downsampled,
                                                       random_state=42)
    
    return features_downsampled, target_downsampled

In [19]:
X_train_down, y_train_down = downsample(X_train, y_train, 0.1)

In [20]:
X_train_down.shape

(21347,)

In [21]:
X_train_down_vect = count_vect.fit_transform(X_train_down)
X_valid_down_vect = count_vect.transform(X_valid)
X_test_down_vect = count_vect.transform(X_test)

In [22]:
print(X_train_down_vect.shape)
print(X_valid_down_vect.shape)
print(X_test_down_vect.shape)

(21347, 45732)
(23894, 45732)
(23894, 45732)


### TF-IDF

In [23]:
count_tf_idf = TfidfVectorizer()
X_train_tf = count_tf_idf.fit_transform(X_train)
X_valid_tf = count_tf_idf.transform(X_valid)
X_test_tf = count_tf_idf.transform(X_test)

In [24]:
(X_train_tf.shape[0] + X_valid_tf.shape[0] + X_test_tf.shape[0])- X.shape[0]

0

In [25]:
print(X_train_tf.shape)
print(X_valid_tf.shape)
print(X_test_tf.shape)

(111504, 138174)
(23894, 138174)
(23894, 138174)


#### Downsample

In [26]:
X_train_down_tf = count_tf_idf.fit_transform(X_train_down)
X_valid_down_tf = count_tf_idf.transform(X_valid)
X_test_down_tf = count_tf_idf.transform(X_test)

In [27]:
print(X_train_down_tf.shape)
print(X_valid_down_tf.shape)
print(X_test_down_tf.shape)

(21347, 45732)
(23894, 45732)
(23894, 45732)


**Вывод:**  
Подтовили данные при помощи CountVectorizer и TfidfVectorizer и подготовили наборы данных на уменьшенной выборке.

## Обучение моделей

### LogisticRegression

Здесь и далее напишем функции, которые будут проводить обучение модели и расчет f1 

In [28]:
def lr(X_train, y_train, X_valid, y_valid, class_weight):
    model = LogisticRegression(random_state=42, class_weight=class_weight)
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    f1 = f1_score(y_valid, pred)
    print(f'Значение f1 логистической регрессии {f1:.2f}')
    return model, f1

#### CountVectorizer

Исходные данные

In [29]:
lr_model_vect1, lr_f1_vect1 = lr(X_train_vect, y_train, X_valid_vect, y_valid, None)

Значение f1 логистической регрессии 0.77


Сбалансированные данные

In [30]:
lr_model_vect2, lr_f1_vect2 = lr(X_train_vect, y_train, X_valid_vect, y_valid, 'balanced')

Значение f1 логистической регрессии 0.76


Уменьшенная выборка

In [31]:
lr_model_vect3, lr_f1_vect3 = lr(X_train_down_vect, y_train_down, X_valid_down_vect, y_valid, None)

Значение f1 логистической регрессии 0.67


#### TfidfVectorizer

Исходные данные

In [32]:
lr_model_tf1, lr_f1_tf1 = lr(X_train_tf, y_train, X_valid_tf, y_valid, None)

Значение f1 логистической регрессии 0.73


Сбалансированные данные

In [33]:
lr_model_tf2, lr_f1_tf2 = lr(X_train_tf, y_train, X_valid_tf, y_valid, 'balanced')

Значение f1 логистической регрессии 0.75


Уменьшенная выборка

In [34]:
lr_model_tf3, lr_f1_tf3 = lr(X_train_down_tf, y_train_down, X_valid_down_tf, y_valid, None)

Значение f1 логистической регрессии 0.64


In [35]:
index = ['CountVectorizer', 'CountVectorizer_balanced', 'CountVectorizer_downsample',
         'TfidfVectorizer', 'TfidfVectorizer_balanced', 'TfidfVectorizer_downsample']
data = {'LogisticRegression': [lr_f1_vect1, lr_f1_vect2, lr_f1_vect3,
                               lr_f1_tf1, lr_f1_tf2, lr_f1_tf3]}
lr_values = pd.DataFrame(index=index, data=data)


Итоговые значения f1 модели LogisticRegression 

In [36]:
lr_values

,LogisticRegression
CountVectorizer,0.767012
CountVectorizer_balanced,0.756386
CountVectorizer_downsample,0.670285
TfidfVectorizer,0.727181
TfidfVectorizer_balanced,0.746410
TfidfVectorizer_downsample,0.643192


### CatBoostClassifier

#### CountVectorizer

Исходные данные

In [37]:
cat_model_vect1 = CatBoostClassifier(random_state=42, eval_metric='F1')
cat_model_vect1.fit(X_train_vect, y_train, eval_set=(X_valid_vect, y_valid), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.101529
0:	learn: 0.4514270	test: 0.4561952	best: 0.4561952 (0)	total: 804ms	remaining: 13m 22s
1:	learn: 0.4032359	test: 0.4029997	best: 0.4561952 (0)	total: 1.46s	remaining: 12m 10s
2:	learn: 0.4341927	test: 0.4413531	best: 0.4561952 (0)	total: 2.15s	remaining: 11m 55s
3:	learn: 0.4597977	test: 0.4639303	best: 0.4639303 (3)	total: 2.79s	remaining: 11m 36s
4:	learn: 0.4787791	test: 0.4828431	best: 0.4828431 (4)	total: 3.44s	remaining: 11m 25s
5:	learn: 0.4311607	test: 0.4279308	best: 0.4828431 (4)	total: 4.08s	remaining: 11m 15s
6:	learn: 0.4851408	test: 0.4891735	best: 0.4891735 (6)	total: 4.72s	remaining: 11m 9s
7:	learn: 0.4845159	test: 0.4888482	best: 0.4891735 (6)	total: 5.38s	remaining: 11m 7s
8:	learn: 0.4733955	test: 0.4845329	best: 0.4891735 (6)	total: 6.05s	remaining: 11m 6s
9:	learn: 0.5036360	test: 0.5125899	best: 0.5125899 (9)	total: 6.69s	remaining: 11m 2s
10:	learn: 0.4895005	test: 0.4939099	best: 0.5125899 (9)	total: 7.34s	remaining: 10m 59s
11:	l

Сбалансированные данные

In [38]:
cat_model_vect2 = CatBoostClassifier(random_state=42, eval_metric='F1', auto_class_weights='Balanced')
cat_model_vect2.fit(X_train_vect, y_train, eval_set=(X_valid_vect, y_valid))

Learning rate set to 0.101529
0:	learn: 0.5196021	test: 0.5227176	best: 0.5227176 (0)	total: 680ms	remaining: 11m 19s
1:	learn: 0.5887671	test: 0.5915967	best: 0.5915967 (1)	total: 1.36s	remaining: 11m 18s
2:	learn: 0.5824812	test: 0.5869660	best: 0.5915967 (1)	total: 2.04s	remaining: 11m 19s
3:	learn: 0.5982209	test: 0.6010312	best: 0.6010312 (3)	total: 2.75s	remaining: 11m 23s
4:	learn: 0.6219314	test: 0.6231107	best: 0.6231107 (4)	total: 3.45s	remaining: 11m 26s
5:	learn: 0.6233221	test: 0.6224758	best: 0.6231107 (4)	total: 4.12s	remaining: 11m 22s
6:	learn: 0.6501911	test: 0.6533333	best: 0.6533333 (6)	total: 4.77s	remaining: 11m 16s
7:	learn: 0.6684702	test: 0.6744492	best: 0.6744492 (7)	total: 5.43s	remaining: 11m 12s
8:	learn: 0.6703310	test: 0.6755212	best: 0.6755212 (8)	total: 6.09s	remaining: 11m 10s
9:	learn: 0.6753923	test: 0.6785409	best: 0.6785409 (9)	total: 6.77s	remaining: 11m 10s
10:	learn: 0.6878282	test: 0.6888604	best: 0.6888604 (10)	total: 7.44s	remaining: 11m 9s
1

Уменьшенная выборка

In [39]:
cat_model_vect3 = CatBoostClassifier(random_state=42, eval_metric='F1', verbose=20)
cat_model_vect3.fit(X_train_down_vect, y_train_down, eval_set=(X_valid_down_vect, y_valid))

Learning rate set to 0.067492
0:	learn: 0.7373190	test: 0.2453372	best: 0.2453372 (0)	total: 202ms	remaining: 3m 21s
20:	learn: 0.7997448	test: 0.3822163	best: 0.3822163 (20)	total: 3.77s	remaining: 2m 55s
40:	learn: 0.8217453	test: 0.4176633	best: 0.4176633 (40)	total: 7.34s	remaining: 2m 51s
60:	learn: 0.8247392	test: 0.6600624	best: 0.6621223 (58)	total: 10.9s	remaining: 2m 47s
80:	learn: 0.8379322	test: 0.6874768	best: 0.6874768 (80)	total: 14.4s	remaining: 2m 43s
100:	learn: 0.8488066	test: 0.6937799	best: 0.6964452 (98)	total: 18s	remaining: 2m 40s
120:	learn: 0.8557609	test: 0.7020617	best: 0.7026428 (109)	total: 21.5s	remaining: 2m 36s
140:	learn: 0.8629451	test: 0.7100917	best: 0.7102597 (137)	total: 25.1s	remaining: 2m 32s
160:	learn: 0.8678812	test: 0.7115070	best: 0.7130562 (146)	total: 28.6s	remaining: 2m 29s
180:	learn: 0.8733533	test: 0.7103983	best: 0.7169259 (175)	total: 32.2s	remaining: 2m 25s
200:	learn: 0.8784523	test: 0.7214912	best: 0.7224557 (199)	total: 35.7s	re

#### TfidfVectorizer

Исходные данные

In [40]:
cat_model_tf1 = CatBoostClassifier(random_state=42, verbose=20, eval_metric='F1')
cat_model_tf1.fit(X_train_tf, y_train, eval_set=(X_valid_tf, y_valid))

Learning rate set to 0.101529
0:	learn: 0.4543516	test: 0.4568591	best: 0.4568591 (0)	total: 1.29s	remaining: 21m 33s
20:	learn: 0.5381818	test: 0.5413937	best: 0.5413937 (20)	total: 25.3s	remaining: 19m 37s
40:	learn: 0.5902141	test: 0.5919589	best: 0.5919589 (39)	total: 48.4s	remaining: 18m 51s
60:	learn: 0.6173991	test: 0.6155562	best: 0.6155562 (59)	total: 1m 11s	remaining: 18m 19s
80:	learn: 0.6404869	test: 0.6431886	best: 0.6431886 (80)	total: 1m 34s	remaining: 17m 52s
100:	learn: 0.6619937	test: 0.6605898	best: 0.6605898 (100)	total: 1m 57s	remaining: 17m 26s
120:	learn: 0.6752214	test: 0.6721617	best: 0.6740859 (118)	total: 2m 20s	remaining: 17m
140:	learn: 0.6911855	test: 0.6831189	best: 0.6831189 (139)	total: 2m 43s	remaining: 16m 35s
160:	learn: 0.7010309	test: 0.6930281	best: 0.6933680 (159)	total: 3m 6s	remaining: 16m 11s
180:	learn: 0.7084091	test: 0.6980352	best: 0.6980352 (180)	total: 3m 29s	remaining: 15m 47s
200:	learn: 0.7182483	test: 0.7047668	best: 0.7050986 (199)	

Сбалансированные данные

In [41]:
cat_model_tf2 = CatBoostClassifier(random_state=42,
                                   verbose=20,
                                   eval_metric='F1',
                                   auto_class_weights='Balanced')
cat_model_tf2.fit(X_train_tf, y_train, eval_set=(X_valid_tf, y_valid))

Learning rate set to 0.101529
0:	learn: 0.5122760	test: 0.5215189	best: 0.5215189 (0)	total: 1.27s	remaining: 21m 14s
20:	learn: 0.7659913	test: 0.7736408	best: 0.7736408 (20)	total: 25.4s	remaining: 19m 43s
40:	learn: 0.8048163	test: 0.8089702	best: 0.8091229 (39)	total: 49.1s	remaining: 19m 7s
60:	learn: 0.8247869	test: 0.8253434	best: 0.8253434 (60)	total: 1m 12s	remaining: 18m 36s
80:	learn: 0.8377229	test: 0.8404927	best: 0.8404927 (80)	total: 1m 36s	remaining: 18m 9s
100:	learn: 0.8484599	test: 0.8524703	best: 0.8524703 (100)	total: 1m 59s	remaining: 17m 43s
120:	learn: 0.8598209	test: 0.8587048	best: 0.8587048 (120)	total: 2m 22s	remaining: 17m 17s
140:	learn: 0.8686854	test: 0.8628691	best: 0.8631256 (139)	total: 2m 46s	remaining: 16m 53s
160:	learn: 0.8751079	test: 0.8683874	best: 0.8683874 (160)	total: 3m 9s	remaining: 16m 28s
180:	learn: 0.8806486	test: 0.8714021	best: 0.8717302 (178)	total: 3m 33s	remaining: 16m 4s
200:	learn: 0.8847461	test: 0.8740297	best: 0.8744793 (194)

Уменьшенная выборка

In [42]:
cat_model_tf3 = CatBoostClassifier(random_state=42, verbose=20, eval_metric='F1')
cat_model_tf3.fit(X_train_down_tf, y_train_down, eval_set=(X_valid_down_tf, y_valid))

Learning rate set to 0.067492
0:	learn: 0.4956677	test: 0.4879068	best: 0.4879068 (0)	total: 342ms	remaining: 5m 41s
20:	learn: 0.8003557	test: 0.3786462	best: 0.5501271 (1)	total: 6.55s	remaining: 5m 5s
40:	learn: 0.8096810	test: 0.6355004	best: 0.6397481 (37)	total: 12.7s	remaining: 4m 55s
60:	learn: 0.8256263	test: 0.6582418	best: 0.6582418 (60)	total: 18.8s	remaining: 4m 49s
80:	learn: 0.8446550	test: 0.6777939	best: 0.6777939 (80)	total: 24.8s	remaining: 4m 41s
100:	learn: 0.8524702	test: 0.6874547	best: 0.6876364 (98)	total: 30.9s	remaining: 4m 35s
120:	learn: 0.8587810	test: 0.6852248	best: 0.6882459 (106)	total: 37s	remaining: 4m 28s
140:	learn: 0.8665646	test: 0.6916919	best: 0.6916919 (140)	total: 43s	remaining: 4m 22s
160:	learn: 0.8730249	test: 0.6971794	best: 0.6971794 (160)	total: 49.2s	remaining: 4m 16s
180:	learn: 0.8783841	test: 0.7017296	best: 0.7030946 (177)	total: 55.2s	remaining: 4m 9s
200:	learn: 0.8857479	test: 0.7046815	best: 0.7055918 (198)	total: 1m 1s	remaini

Значения f1 модели CatBoost  

Данные| F1| iteration|
:-----|:--|:---------|
CountVectorizer| 0.72| 882|
CountVectorizer_balanced| 0.88| 940|
CountVectorizer_downsample| 0.71| 375|
TfidfVectorizer| 0.75| 907|
TfidfVectorizer_balanced| 0.89| 935|
TfidfVectorizer_downsample| 0.71| 428|

### LightGBMClissifier

In [43]:
def lgb_func(X_train, y_train, X_valid, y_valid, class_weight):
    model = lgb.LGBMClassifier(objective="binary", 
                               random_state=42, 
                               verbosity=20,
                              class_weight=class_weight)
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    f1 = f1_score(y_valid, pred)
    return model, f1

#### CountVectorizer

Исходные данные

In [54]:
lgb_model_vect1, lgb_f1_vect1 = lgb_func(X_train_vect.astype('float'),
                                         y_train, 
                                         X_valid_vect.astype('float'), 
                                         y_valid, None)

[LightGBM] [Info] Number of positive: 11330, number of negative: 100174
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.997644
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.997644
[LightGBM] [Debug] init for col-wise cost 1.841167 seconds, init for row-wise cost 1.955342 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.962122 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46590
[LightGBM] [Info] Number of data points in the train set: 111504, number of used features: 10695
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101611 -> initscore=-2.179455
[LightGBM] [Info] Start training from score -2.179455
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 30
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 30
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 30
[LightGBM] [Debug] Trained a tree with lea

In [55]:
print(f'Значение f1 {lgb_f1_vect1:.2f}')

Значение f1 0.75


Сбалансированные данные

In [56]:
lgb_model_vect2, lgb_f1_vect2 = lgb_func(X_train_vect.astype('float'),
                                         y_train, 
                                         X_valid_vect.astype('float'), 
                                         y_valid, 'balanced')

[LightGBM] [Info] Number of positive: 11330, number of negative: 100174
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.997644
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.997644
[LightGBM] [Debug] init for col-wise cost 1.974267 seconds, init for row-wise cost 1.890505 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.981207 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 46590
[LightGBM] [Info] Number of data points in the train set: 111504, number of used features: 10695
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 25


In [57]:
print(f'Значение f1 {lgb_f1_vect2:.2f}')

Значение f1 0.76


Уменьшенная выборка

In [62]:
lgb_model_vect3, lgb_f1_vect3 = lgb_func(X_train_down_vect.astype('float'), 
                                         y_train_down,
                                         X_valid_down_vect.astype('float'),
                                         y_valid, None)

[LightGBM] [Info] Number of positive: 11330, number of negative: 10017
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.994612
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.994612
[LightGBM] [Debug] init for col-wise cost 0.109674 seconds, init for row-wise cost 0.109748 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.111522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 13462
[LightGBM] [Info] Number of data points in the train set: 21347, number of used features: 3460
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.530754 -> initscore=0.123170
[LightGBM] [Info] Start training from score 0.123170
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 24
[Li

In [63]:
print(f'Значение f1 {lgb_f1_vect3:.2f}')

Значение f1 0.72


#### TfidfVectorizer

Исходные данные

In [64]:
lgb_model_tf1, lgb_f1_tf1 = lgb_func(X_train_tf, y_train, X_valid_tf, y_valid, None)

[LightGBM] [Info] Number of positive: 11330, number of negative: 100174
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.997644
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.997644
[LightGBM] [Debug] init for col-wise cost 1.931295 seconds, init for row-wise cost 1.909262 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.054442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 569538
[LightGBM] [Info] Number of data points in the train set: 111504, number of used features: 10695
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101611 -> initscore=-2.179455
[LightGBM] [Info] Start training from score -2.179455
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 30
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 

In [65]:
print(f'Значение f1 {lgb_f1_tf1:.2f}')

Значение f1 0.76


Сбалансированные данные

In [66]:
lgb_model_tf2, lgb_f1_tf2 = lgb_func(X_train_tf, y_train, X_valid_tf, y_valid, 'balanced')

[LightGBM] [Info] Number of positive: 11330, number of negative: 100174
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.997644
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.997644
[LightGBM] [Debug] init for col-wise cost 1.871253 seconds, init for row-wise cost 1.990715 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.993854 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 569538
[LightGBM] [Info] Number of data points in the train set: 111504, number of used features: 10695
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 25
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 26

In [67]:
print(f'Значение f1 {lgb_f1_tf2:.2f}')

Значение f1 0.75


Уменьшенная выборка

In [68]:
lgb_model_tf3, lgb_f1_tf3 = lgb_func(X_train_down_tf, y_train_down, X_valid_down_tf, y_valid, None)

[LightGBM] [Info] Number of positive: 11330, number of negative: 10017
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.994612
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.994612
[LightGBM] [Debug] init for col-wise cost 0.153387 seconds, init for row-wise cost 0.129377 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.172415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 123137
[LightGBM] [Info] Number of data points in the train set: 21347, number of used features: 3460
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.530754 -> initscore=0.123170
[LightGBM] [Info] Start training from score 0.123170
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 26
[L

In [69]:
print(f'Значение f1 {lgb_f1_tf3:.2f}')

Значение f1 0.72


In [70]:
index = ['CountVectorizer', 'CountVectorizer_balanced', 'CountVectorizer_downsample',
         'TfidfVectorizer', 'TfidfVectorizer_balanced', 'TfidfVectorizer_downsample']
data = {'f1': [lgb_f1_vect1, lgb_f1_vect2, lgb_f1_vect3,
               lgb_f1_tf1, lgb_f1_tf2, lgb_f1_tf3]}
lgb_values = pd.DataFrame(data=data, index=index)

Значения f1 на модели LightGBM

In [71]:
lgb_values

,f1
CountVectorizer,0.749880
CountVectorizer_balanced,0.756593
CountVectorizer_downsample,0.719480
TfidfVectorizer,0.755992
TfidfVectorizer_balanced,0.749577
TfidfVectorizer_downsample,0.719958


**Вывод:**  
Самое высокое значение f1 (0.89) у модели CatBoost на данных сбалансированных и преобразованных с помощью TfidfVectorizer. Ее и протестируем

## Тестирование

### CatBoost

In [72]:
total_predict = cat_model_tf2.predict(X_test_tf)
total_f1 = f1_score(y_test, total_predict)
total_f1

0.7682346219767663

**Вывод:**  
На тесте модель показала значение f1 0.76, что выше значения в поставленной задаче (0.75)